**AMAZON MACHINE LEARNING WITH PYTHON**

<hr>
This notebook is about playing the content of : http://docs.aws.amazon.com/fr_fr/machine-learning/latest/dg/tutorial.html
<hr>

In [ ]:
!echo $USER-aws-bigdata-workshop-us-east-1  

**IMPORT THE DATASET**

In [ ]:
!mkdir ML

In [ ]:
!aws s3 cp s3://aws-potus-eu-west-1/ML/ ML/ --recursive --only-show-errors

In [ ]:
import pandas,os
df = pandas.read_csv('ML/banking.csv', sep=',',error_bad_lines=False)
df.head()

**USE AMAZON MACHINELEARNING**

In [ ]:
import boto3

In [ ]:
ml = boto3.client('machinelearning',region_name='eu-west-1')

It's needed to add permissions allow Amazon ML to access your bucket. 
Please follow the documentation : http://docs.aws.amazon.com/fr_fr/machine-learning/latest/dg/granting-amazon-ml-permissions-to-read-your-data-from-amazon-s3.html

In [ ]:
ds = ml.create_data_source_from_s3(DataSourceId= "datasource-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"),\
        DataSpec= {"DataLocationS3":"s3://aws-potus-eu-west-1/ML/banking.csv",\
        "DataSchemaLocationS3":"s3://aws-potus-eu-west-1/ML/banking.schema"},ComputeStatistics = True)

In [ ]:
print(ml.get_data_source(DataSourceId= "datasource-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP")))

In [ ]:
model = ml.create_ml_model(
   MLModelId="model-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"), 
   MLModelType="BINARY", 
   TrainingDataSourceId="datasource-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"))

In [ ]:
print(ml.get_ml_model(MLModelId="model-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP")))

<hr>
Wait for **5 minute** in order to have Datasource and Model available. You can check the availability of the model with previous command : print(ml.get_ml_model('model'))
<hr>

**Time to evaluate the model**

In [ ]:
ml.create_evaluation(EvaluationId="evaluation-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"),\
                    EvaluationName="evaluation-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"),\
                    MLModelId="model-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"),\
                    EvaluationDataSourceId="datasource-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"))

In [ ]:
ml.get_evaluation(EvaluationId="evaluation-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"))

**Time to use the model**

In [ ]:
ml.create_realtime_endpoint(MLModelId="model-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"))

**Time to do a prediction**

In [ ]:
prediction = ml.predict(MLModelId="model-"+os.getenv("USER")+"-"+os.getenv("WORKSHOP"),\
        PredictEndpoint='https://realtime.machinelearning.eu-west-1.amazonaws.com',\
            Record={'age':'32','job':'none','marital':'married','education':'university',})
print(prediction)